In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_model = pd.read_csv('Data\Crop_Yield_Cleaned.csv')

In [3]:
x = df_model.drop(['Yield_tons_per_hectare'],axis=1)
y = df_model['Yield_tons_per_hectare']

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=44)
from sklearn.preprocessing import StandardScaler,Normalizer
# stdscaler = StandardScaler()
# nrml = Normalizer()
# X_train = stdscaler.fit_transform(x_train)
# X_test = stdscaler.transform(x_test)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [6]:
##Create a Function to Evaluate Model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

Initial Model Training

In [ ]:
## Beginning Model Training
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "ElasticNet":ElasticNet(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Adaboost Regressor":AdaBoostRegressor(),
    "Graident BoostRegressor":GradientBoostingRegressor(),

   
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_train_adjusted_r_squared = round(1 - (1-model_train_r2)*(len(y)-1)/(len(y)-x.shape[1]-1),6)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    model_test_adjusted_r_squared = round(1 - (1-model_test_r2)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print("Adjusted R2 Score : {:.4f}".format(model_train_adjusted_r_squared))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    print("Adjusted R2 Score : {:.4f}".format(model_test_adjusted_r_squared))
    print('='*35)
    print('\n')

Hyperparameter Tuning


In [8]:
rf_params = {'criterion':['squared_error', 'absolute_error', 'poisson', 'friedman_mse'],
             "max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]
            }




In [9]:
# Models list for Hyperparameter tuning
cv_models = [
    
                   ("RF", RandomForestRegressor(), rf_params),
                   
]


Random CV Hyperparameter tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in cv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

In [ ]:
## Retraining the models with best parameters with RandomCV
models = {
    "Random Forest Regressor": RandomForestRegressor(n_estimators= 500, min_samples_split= 8, max_features= 8, max_depth= None, criterion= 'squared_error'),  
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_train_adjusted_r_squared = round(1 - (1-model_train_r2)*(len(y)-1)/(len(y)-x.shape[1]-1),6)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    model_test_adjusted_r_squared = round(1 - (1-model_test_r2)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print("Adjusted R2 Score : {:.4f}".format(model_train_adjusted_r_squared))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    print("Adjusted R2 Score : {:.4f}".format(model_test_adjusted_r_squared))
    print('='*35)
    print('\n')

Grid CV


In [12]:
from sklearn.model_selection import GridSearchCV

model_param = {}
for name, model, params in cv_models:
    random = GridSearchCV(estimator=model,
                                   param_grid=params,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

In [ ]:
## Retraining the models with best parameters of Grid CV
models = {
    "Random Forest Regressor": RandomForestRegressor(n_estimators= 500, min_samples_split= 8, max_features= 8, max_depth= None, criterion= 'squared_error'),
    # "GradientBoost":GradientBoostingRegressor(n_estimators= 500, min_samples_split= 20, max_depth=5, loss='squared_error', learning_rate= 0.01, criterion='friedman_mse'),
   
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')

#Observations
Linear Regression, Ridge, Random Forest, Adaboost, Gradient Boost, XGboost Models Performed Exceptionally well in Initial Evaluation - with best Adjusted R2 Score - 91% with Linear Regression, Ridge and Gradient Boost

To further Enhanced the performance we Hyperparameter tuned the Model with Gradient Boost and Xgboost - Using Random and Grid CV 

Using XGBoostRegressor for final Predictions


======================================================================================
Pickling the XGB Regressor and StandardScalar

In [ ]:
#Pickling the XGBoost Model

Ridge_Model = Ridge()
Ridge_Model.fit(x_train, y_train) # Train model

In [12]:
import pickle

# Assuming your trained model is stored in the variable 'model'


pickle.dump(Ridge_Model,open('Ridge_Model_Agriculture_Yield_Prediction.pkl', 'wb'))

pickle.dump(stdscaler,open('Scalar_Model_Agriculture_Yield_Prediction.pkl', 'wb'))


